# XUV3XO TGDi — CAN Log Decoder (Minimal)

This notebook parses raw CAN logs, extracts UDS `0x62` responses, maps DIDs using the CSV, applies scaling, and plots signals.

In [ ]:

import pandas as pd, re
from pathlib import Path

# === Configuration ===
LOG_PATH = Path("../logs/sample.csv")   # Replace with your file (CSV with columns: time, can_id, data)
PID_CSV = Path("../pids/xuv3xo_pids_with_meanings.csv")

# Load PID table
pids = pd.read_csv(PID_CSV)
pids['DID_int'] = pids['DID'].apply(lambda x: int(x,16))

print("Loaded PIDs:", len(pids))
pids.head()


In [ ]:

# === Helper: parse a CSV log with columns time,can_id,data (hex bytes space- or no-space-separated) ===
def parse_hex_bytes(s):
    s = s.strip().replace(" ", "")
    return [int(s[i:i+2],16) for i in range(0, len(s), 2)]

df = pd.read_csv(LOG_PATH)
df['can_id'] = df['can_id'].apply(lambda x: int(str(x),16) if isinstance(x,str) else int(x))
df['bytes'] = df['data'].apply(parse_hex_bytes)

print(df.head())
print("Frames:", len(df))


In [ ]:

# === Extract UDS positive responses (0x62) from 0x7E8 ===
resp = df[(df['can_id']==0x7E8) & (df['bytes'].apply(lambda b: len(b)>=3 and b[0]==0x62))].copy()

def did_from_bytes(b):
    return (b[1]<<8) | b[2]

resp['DID_int'] = resp['bytes'].apply(did_from_bytes)
resp = resp.merge(pids[['DID_int','Name','Scaling','Unit','Protected','Meaning']], on='DID_int', how='left')

print("UDS responses decoded:", len(resp))
resp[['time','can_id','DID_int','Name','Meaning']].head(10)


In [ ]:

# === Apply simple scalings for common types ===
import math

def apply_scaling(row):
    b = row['bytes']
    did = row['DID_int']
    # data start after [62, DID_HI, DID_LO]
    data = b[3:]
    # Default raw value heuristic: 1 or 2 bytes
    raw = None
    if len(data)>=2:
        raw = (data[0]<<8)|data[1]
    elif len(data)>=1:
        raw = data[0]
    else:
        return None

    # Specific DIDs with known scaling
    if did == 0x03E9:  # RPM
        return raw/4.0
    if did == 0x03EA:  # Speed
        return float(raw)
    if did == 0x03F0:  # MAP kPa abs
        return raw*0.5
    if did == 0x03EB or did == 0x03EC:  # temps
        return raw-40.0

    # Fallback: return raw
    return float(raw)

resp['Value'] = resp.apply(apply_scaling, axis=1)
resp_simple = resp[['time','DID_int','Name','Value','Unit','Protected','Meaning']].dropna()
resp_simple.head(20)


In [ ]:

# === Plot a couple of key signals ===
import matplotlib.pyplot as plt

def plot_signal(did, title):
    d = resp_simple[resp_simple['DID_int']==did]
    if d.empty: 
        print(f"No data for DID 0x{did:04X}")
        return
    plt.figure()
    plt.plot(d['time'].values, d['Value'].values)
    plt.title(title)
    plt.xlabel("time")
    plt.ylabel(title)
    plt.show()

plot_signal(0x03E9, "Engine RPM")
plot_signal(0x03F0, "MAP (kPa abs)")
plot_signal(0x03EA, "Vehicle Speed (km/h)")
